In [1]:
import numpy as np
import qiskit

In [2]:
num_qubits = 5

pauli_type = ['I','X','Y','Z']
I = np.eye(2)
X = np.array([[0,1+0.j],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1+0.j,0],[0,-1]])
gate_dict = {'X': X, 'Y': Y, 'Z': Z}

L_str_list = []
L_mat_list = []

# Single qubit op
for n in range(num_qubits):
    for pauli in ['X','Y','Z']:
        L_str_list.append('I'*n + pauli + 'I'*(num_qubits-n-1))
        mat = np.kron(np.kron(np.eye(2**n), gate_dict[pauli]), np.eye(2**(num_qubits-n-1)))
        L_mat_list.append(mat)
        
# Two-local op
for n in range(num_qubits):
    for pauli1 in ['X','Y','Z']:
        for pauli2 in ['X','Y','Z']:
            gate_str = list('I'*num_qubits)
            gate_str[n] = pauli1
            gate_str[(n+1) % num_qubits] = pauli2
            L_str_list.append(''.join(gate_str))
            
            if n != num_qubits - 1:
                mat = np.kron(np.kron(np.kron(np.eye(2**n), gate_dict[pauli1]), gate_dict[pauli2]), np.eye(2**(num_qubits-n-2)))
            else:
                mat = np.kron(np.kron(gate_dict[pauli2], np.eye(2**(num_qubits - 2))), gate_dict[pauli1])
            L_mat_list.append(mat)

In [3]:
L_dict = dict(zip(L_str_list, L_mat_list))

coeffs = np.random.rand(len(L_dict))
norm = np.linalg.norm(coeffs)
coeffs = coeffs / norm

H = [(L_str, coeffs[i]) for i,L_str in enumerate(L_dict.keys())]

H_mat = 0 + 0.j
for H_comp,coeff in H:
    H_mat += coeff * L_dict[H_comp]

In [4]:
H_eigvals, H_eigvecs = np.linalg.eigh(H_mat)

ground_energy = H_eigvals[0]
v = H_eigvecs[:,0]

print(ground_energy)

-1.6983110273752013


In [5]:
def correlation_matrix(L_dict, v):
    m = len(L_dict)
    M = np.zeros((m,m), dtype='complex128')
    L_mean_list = np.array([v.conj().T @ L @ v for L in L_dict.values()])
    
    
    for i, L_i in enumerate(L_dict.values()):
        L_i_mean = L_mean_list[i]
        
        for j, L_j in enumerate(L_dict.values()):
            if j >= i:
                L_j_mean = L_mean_list[j]
                anticom = L_i @ L_j + L_j @ L_i
                anticom_mean = v.conj().T @ anticom @ v
                M[i,j] = anticom_mean/2. - L_i_mean * L_j_mean
            else:
                M[i,j] = M[j,i]    
    
    return M

In [6]:
M = correlation_matrix(L_dict, v)
M_eigvals, M_eigvecs = np.linalg.eigh(M)

## Note: M is positive semidefinite
eps = 1e-8
print('Number of small eigenvalues: ', (M_eigvals < eps).sum())
print('Small eigenvalues: ', M_eigvals[M_eigvals < eps])

H_recon = 0 + 0.0j
for i,w in enumerate(M_eigvecs[:,0]):
    H_recon += w * L_mat_list[i]

Number of small eigenvalues:  1
Small eigenvalues:  [-5.97534592e-15]


In [7]:
H_mat

array([[ 0.96053759+0.j        ,  0.58066127-0.26044284j,
         0.16567653-0.28336413j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.58066127+0.26044284j,  0.49139192+0.j        ,
         0.29304984-0.04898994j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.16567653+0.28336413j,  0.29304984+0.04898994j,
         0.14368897+0.j        , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       ...,
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ..., -0.11442464+0.j        ,
         0.29304984-0.04898994j,  0.13002459+0.16795539j],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ...,  0.29304984+0.04898994j,
         0.08034905+0.j        , -0.18736443+0.20644509j],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j       

In [8]:
H_recon

array([[ 0.96053759+8.61357097e-13j,  0.58066127-2.60442840e-01j,
         0.16567653-2.83364131e-01j, ...,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.58066127+2.60442840e-01j,  0.49139192+5.04119292e-13j,
         0.29304984-4.89899400e-02j, ...,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.16567653+2.83364131e-01j,  0.29304984+4.89899400e-02j,
         0.14368897-2.61822038e-13j, ...,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       ...,
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, ..., -0.11442464-1.37038204e-13j,
         0.29304984-4.89899400e-02j,  0.13002459+1.67955393e-01j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, ...,  0.29304984+4.89899400e-02j,
         0.08034905+3.91495880e-13j

In [9]:
rec_eigvals, rec_eigvecs = np.linalg.eigh(H_recon)

In [10]:
H_eigvals

array([-1.69831103, -1.58394874, -1.48706712, -1.35567132, -1.23965952,
       -1.15012007, -1.02670666, -0.79702462, -0.7623722 , -0.55776998,
       -0.53520235, -0.45080295, -0.20286308, -0.13696942, -0.0543655 ,
        0.03603221,  0.11358719,  0.20377127,  0.2756023 ,  0.42961719,
        0.46825192,  0.49175263,  0.56651968,  0.62087751,  0.67918538,
        0.74403483,  0.83982964,  1.03734087,  1.13129602,  1.22768201,
        1.45909794,  2.71437599])

In [11]:
rec_eigvals

array([-1.69831103, -1.58394874, -1.48706712, -1.35567132, -1.23965952,
       -1.15012007, -1.02670666, -0.79702462, -0.7623722 , -0.55776998,
       -0.53520235, -0.45080295, -0.20286308, -0.13696942, -0.0543655 ,
        0.03603221,  0.11358719,  0.20377127,  0.2756023 ,  0.42961719,
        0.46825192,  0.49175263,  0.56651968,  0.62087751,  0.67918538,
        0.74403483,  0.83982964,  1.03734087,  1.13129602,  1.22768201,
        1.45909794,  2.71437599])